In [191]:
import pandas as pd
df = pd.read_csv("/home/ethereum/Downloads/check.csv")

In [192]:
column = []
for i in range(df.shape[0]):
    column.append("no")
print(column)

['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no']


In [193]:
df["o_freedelivery"] = column
df.head()

,o_orderkey,o_custkey,o_totalprice,o_orderdate,o_freedelivery
0,23015,548,94748.05,1992-01-12,no
1,38310,548,79316.37,1992-04-15,no
2,17632,548,191202.12,1993-02-22,no
3,34435,548,40953.07,1994-08-06,no
4,35270,548,27725.67,1994-11-10,no


In [194]:
df.iloc[0:1].shape

(1, 5)

In [207]:
def check_freedelivery(total_spent, o_totalprice):
    threshold = 200000
    if (total_spent + float(o_totalprice) >= threshold):
        return "yes"
    else:
        return "no"

def calc_total(o_custkey, o_orderdate, chain):
    total_spent = 0
    for i in range(chain.shape[0]-1,-1,-1):
        tx_date = chain.iloc[i, 3]
        spent_time = int(o_orderdate[:4]) - int(tx_date[:4]) + int(o_orderdate[5:7])/12 - int(tx_date[5:7])/12 + int(o_orderdate[8:])/365 - int(tx_date[8:])/365
        tx_price = round(chain.iloc[i, 2])
        tx_custkey = chain.iloc[i, 1]
        tx_freedelivery = chain.iloc[i, 4]
            
        if spent_time <= 1:
            if o_custkey == tx_custkey:
                if tx_freedelivery == "no":
                    total_spent += tx_price
                else:
                    return total_spent
        else:
            return total_spent
    return total_spent

In [208]:
new_df = df.iloc[:2]
print(new_df.shape)
print(new_df.iloc[0][3])

(2, 5)
1992-01-12


/tmp/ipykernel_4044/4185693459.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(new_df.iloc[0][3])


In [197]:
total = []
rewards = []
for tx_no in range(df.shape[0]):
    total_spent = calc_total(df.iloc[tx_no, 1], df.iloc[tx_no, 3], df.iloc[:tx_no])
    print(total_spent, end= "=>")
    rewards.append(check_freedelivery(total_spent, df.iloc[tx_no, 2]))
    print(check_freedelivery(total_spent, df.iloc[tx_no, 2]))

0=>no
94748=>no
79316=>yes
0=>no
40953=>no
0=>no
186621=>yes
254101=>yes
546780=>yes
0=>no


In [198]:
df["o_freedelivery"] = rewards

In [199]:
print(total)

[]


In [200]:
df.to_csv("newcheck.csv")